# Data extraction notebook

This notebook will be used to extract the data from the ohio runs and potentially the data generated by geant

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import pandas as pd
from collections import defaultdict

### Creates a list of file paths of the original ohio data

In [2]:
run_path = 'ohio_data/runs/'
run_paths = [os.path.join(run_path, file) for file in os.listdir('ohio_data/runs/')
             if file.endswith('.txt') and file != 'help.txt']

### Creates a dictionary holding each file and the energy deposition of each channel

In [3]:
file_edeps = {}

for file_path in run_paths:
    file_name = os.path.basename(file_path)
    df = pd.read_csv(file_path, delimiter='\t', skiprows=19)
    
    channel_edep = {}
    for col in df.columns[5:-2]:
        #print(col, df[col].sum(), type(df[col].sum()))
        channel_edep[col] = df[col].sum()
    
    file_edeps[file_name] = channel_edep

In [10]:
run_1 = file_edeps['R01.txt'].values()
print(len(run_1))
plt.imshow(run_1.reshape(8, 8))

64


AttributeError: 'dict_values' object has no attribute 'reshape'

### Saves the channel and the corresponding energy deposition in a file with the same name as the original data

In [4]:
for file in file_edeps:
    with open(f'ohio_data/extracted_data/{file}', 'w') as fhandle:
        print('Channel edep', file=fhandle)
        
        for channel, edep in file_edeps[file].items():
            print(f"{channel.replace(' ','')} {edep}", file=fhandle)